# Lecture 1 动态规划

## 什么是动态规划

动态规划核心思想: **通过存储中间结果, 减少递归程序中的重复计算, 从而提高程序的效率**。

重点:

1. 观察变化，抽象出 **状态空间**
2. 思考平凡解, 确定 dp base
3. 多做简单尝试，思考状态转移方程, 确定 dp sweep

e.g. 斐波那契数列

下面将给出两个版本的斐波那契数列的实现, 第一个版本是暴力递归实现, 第二个版本是存储中间结果的递归实现。

版本一: Fibbiacci数列的递归实现

以下的这个实现没有存储任何结果, 会导致重复计算, 效率低下。

In [4]:
# 暴力递归实现
# 时间复杂度: O(2^n)
def fib(n):
    if n == 1 or n == 2:
        return 1
    return fib(n-1) + fib(n-2)

In [17]:
%%time
fib(35);

CPU times: total: 1.78 s
Wall time: 1.79 s


第二个版本的实现, 存储了中间结果。

相比于 version 1, version 2 的时间复杂度大大降低。但是存储的数据结构是list, 会导致存储空间的浪费。

因此动态规划: 以 **空间换时间**

In [11]:
# 递归实现, 存储中间结果
# 时间复杂度: O(n)
def fib_dp(n):
    fib_dict = {}
    fib_dict[1] = 1
    fib_dict[2] = 1
    for i in range(3, n+1):
        fib_dict[i] = fib_dict[i-1] + fib_dict[i-2]
    return fib_dict[n]

In [18]:
%%time
fib_dp(3500);

CPU times: total: 15.6 ms
Wall time: 1e+03 µs


### 题目一:

![ex1](./imgs/q1.png)

我们首先来看一下暴力递归的实现, 代码如下:

**version 1**: 暴力递归

In [40]:
def wander_count(N, begin, dest, steps):
    return wander_count_recursive(N, begin, dest, steps)

def wander_count_recursive(N, begin, dest, steps):
    if steps == 0:
        return dest == begin
    if begin == 1:
        return wander_count_recursive(N, begin+1, dest, steps-1)
    if begin == N:
        return wander_count_recursive(N, begin-1, dest, steps-1)
    return wander_count_recursive(N, begin+1, dest, steps-1) + wander_count_recursive(N, begin-1, dest, steps-1)

wander_count(4, 2, 4, 6)

8

第一次优化:

递归的返回值究竟由哪些参数决定？？ ==> 状态空间选择

状态空间 key: (begin, steps)

因此 dp 数组的形状应当是: range(begin) x range(steps)

**出现重复解的递归可以优化**

**version 2**: 第一次优化 **silly cache**

又名 **自顶向下的动态规划**, **记忆化搜索**

In [41]:
def wander_count_v2(N, begin, dest, steps):
    # begin: 1, 2, ..., N
    # steps: 0, 1, ..., steps
    # dp[i][j] means begin=i+1, steps=j
    dp = [[-1] * (steps+1) for _ in range(N)]
    return wander_count_recursive_v2(N, begin, dest, steps, dp)

def wander_count_recursive_v2(N, begin, dest, steps, dp):
    # 如果已经有结果，那么直接返回
    if dp[begin-1][steps] != -1:
        return dp[begin-1][steps]
    # 否则, 计算结果并且存在 dp 里
    if steps == 0:
        ans = dest == begin
    elif begin == 1:
        ans = wander_count_recursive_v2(N, begin+1, dest, steps-1, dp)
    elif begin == N:
        ans = wander_count_recursive_v2(N, begin-1, dest, steps-1, dp)
    else:
        ans = wander_count_recursive_v2(N, begin+1, dest, steps-1, dp) + wander_count_recursive_v2(N, begin-1, dest, steps-1, dp)
    dp[begin-1][steps] = ans
    return ans

wander_count_v2(4, 2, 4, 6)

8

第二次优化

我们不再简单地从树顶开始搜索，而是考虑 dynamic。

In [42]:
# 简单定义一个工具函数, 用于打印二维数组
def print_2d_array(arr_2d):
    for row in arr_2d:
        row = [str(el) for el in row]
        print(", ".join(row))

In [43]:
def wander_count_v3(N, begin, dest, steps):
    # begin: 1, 2, ..., N
    # steps: 0, 1, ..., steps
    # dp[i][j] means begin=i+1, steps=j
    dp = [[-1] * (steps+1) for _ in range(N)]
    # dp base, 对应于原始递归中的 recursive base
    for i in range(N):
        dp[i][0] = int(i == dest-1)

    print_2d_array(dp)
    # dp sweep
    for k in range(1, steps+1):
        # # version 1: 循环内判断
        # for i in range(N):
        #     # 计算 dp[i][k]
        #     ans = 0
        #     if i > 0:
        #         ans += dp[i-1][k-1]
        #     if i < N-1:
        #         ans += dp[i+1][k-1]
        #     dp[i][k] = ans
        # version 2: 循环外判断
        dp[0][k] = dp[1][k-1]
        for i in range(1, N-1):
            dp[i][k] = dp[i-1][k-1] + dp[i+1][k-1]
        dp[N-1][k] = dp[N-2][k-1]
    print_2d_array(dp)

    return dp[begin-1][steps]

wander_count_v3(4, 2, 4, 6)

0, -1, -1, -1, -1, -1, -1
0, -1, -1, -1, -1, -1, -1
0, -1, -1, -1, -1, -1, -1
1, -1, -1, -1, -1, -1, -1
0, 0, 0, 1, 0, 3, 0
0, 0, 1, 0, 3, 0, 8
0, 1, 0, 2, 0, 5, 0
1, 0, 1, 0, 2, 0, 5


8

### 题目三

![](./imgs/q2.png)

In [88]:
arr = [10, 100, 20, 50]

def win_game(arr):
    return max(win_game_recursive(arr, 0, len(arr)-1, True), win_game_recursive(arr, 0, len(arr)-1, False))

def win_game_recursive(arr, start, end, first):
    # recursive base
    if start == end:
        return arr[start] if first else 0

    # recursive update
    if first:
        return max(win_game_recursive(arr, start+1, end, not first) + arr[start], win_game_recursive(arr, start, end-1, not first) + arr[end])
    else:
        return min(win_game_recursive(arr, start+1, end, not first), win_game_recursive(arr, start, end-1, not first))
win_game(arr)

150

改写成动态规划

In [86]:
def win_game(arr):
    # dp[start][end][first]
    dp = [[[-1] * 2 for _ in range(len(arr))] for _ in range(len(arr))]
    # dp base
    for i in range(len(arr)):
        dp[i][i][1] = arr[i]
        dp[i][i][0] = 0
    # dp sweep
    for i in range(len(arr)-1):
        for j in range(len(arr)-i-1):
            dp[j][j+i+1][0] = min(dp[j+1][j+i+1][1], dp[j][j+i][1])
            dp[j][j+i+1][1] = max(dp[j+1][j+i+1][0] + arr[j], dp[j][j+i][0] + arr[i+j+1])
    return dp[0][len(arr)-1][1]

win_game(arr)

150